In [1]:
import numpy as np
import pandas as pd
import tensorflow
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
tensorflow.keras.__version__

'2.4.0'

In [2]:
file = 'cleaned_data_t.csv'
clean_df = pd.read_csv(file)

In [3]:
print(clean_df.count())
clean_df.isnull().sum().sum()

Unnamed: 0              1401
Self-employed           1401
Tech_employee           1401
Current_disorder        1401
Condition               1401
Gender                  1400
Remote                  1401
Anxiety Disorder        1401
Mood Disorder           1401
Eating Disorder         1401
ADHD                    1401
Personality Disorder    1401
Other                   1401
dtype: int64


1

In [4]:
#clean_df = clean_df[["Self-employed", "Tech_employee", "Gender", "Remote", "Current_disorder", "Anxiety Disorder", "Mood Disorder", "Eating Disorder", "ADHD", "Personality Disorder", "Other"]]
cleaned_df = clean_df[["Self-employed", "Tech_employee", "Gender", "Remote", "Current_disorder"]]

In [5]:
cleaned_df.head()
cleaned_df["Gender"] = cleaned_df["Gender"].fillna(0)
cleaned_df.isnull().sum()

C:\Users\tonyg\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Self-employed       0
Tech_employee       0
Gender              0
Remote              0
Current_disorder    0
dtype: int64

In [6]:
cleaned_df.head()
cleaned_df= cleaned_df.astype(float)
cleaned_df.dtypes

Self-employed       float64
Tech_employee       float64
Gender              float64
Remote              float64
Current_disorder    float64
dtype: object

In [7]:
data = cleaned_df.values
X = data[:, 0:4]
y = data[:, 4]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler().fit(X_train)

In [9]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [11]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [12]:
cleaned_df.isnull().sum().sum()
#clean_df

0

In [13]:
model = Sequential()
number_inputs = 4
number_hidden_nodes = 12
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [14]:
number_classes = 1
model.add(Dense(units=100, activation='relu', input_dim=20))
#model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                60        
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1300      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 1,562
Trainable params: 1,562
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
from keras.optimizers import SGD
opt = SGD(lr=.01)
#model.compile(loss = "binary_crossentropy", optimizer = opt,metrics=['accuracy'])
#'categorical_crossentropy'

In [ ]:
tensorflow.config.run_functions_eagerly(True)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100


C:\Users\tonyg\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


33/33 - 0s - loss: 0.6839 - accuracy: 0.5790
Epoch 2/100
33/33 - 0s - loss: 0.6661 - accuracy: 0.6295
Epoch 3/100
33/33 - 0s - loss: 0.6602 - accuracy: 0.6286


In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
#check all combinations to see what comes out positive
all_answer_combinations = np.array([
    [0, 0, 0, 0], 
    [0, 0, 0, 1], 
    [0, 0, 1, 0],
    [0, 0, 1, 1],
    [0, 1, 0, 0],
    [0, 1, 0, 1],
    [0, 1, 1, 0],
    [0, 1, 1, 1],
    [1, 0, 0, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [1, 1, 0, 1],
    [1, 1, 1, 0],
    [1, 1, 1, 1],
    [1, 0, 0, 0],
    [0, 0, 0, 0.5], 
    [0, 0, 1, 0.5],
    [0, 1, 0, 0.5],
    [0, 1, 1, 0.5],
    [1, 0, 0, 0.5],
    [1, 0, 1, 0.5],
    [1, 1, 0, 0.5],
    [1, 1, 1, 0.5]])
for i in all_answer_combinations:
    data = np.array([i])
    if model.predict_classes(data) != 0:
        print(f"Array: {data} Predicted class: {model.predict_classes(data)}")

In [ ]:
with open(f"{cleaned_df.columns[4]}_predictions.csv", mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Selection',cleaned_df.columns[4], 'Model Loss', 'Model Accuracy'])
    for i in all_answer_combinations:
        data = np.array([i])
        #if model.predict_proba(data) != 0:
        number = '%.2f' % round(model.predict_proba(data)[0][1],2)
        print(f"Array: {data} Predicted class: {number}")
        writer.writerow([data, number, model_loss, model_accuracy])
